In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('com.spark-analysis').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 17:34:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [69]:
FILE_NAME = 'skills-data%20Engineer%20india.json'

In [46]:
skills_df.head(5)

[Row(chips=[Row(options=[Row(text='All', value=None), Row(text='Data engineer', value='data engineer'), Row(text='Engineering', value='engineering'), Row(text='Engineer', value='engineer'), Row(text='Software engineer', value='software engineer'), Row(text='Data engineering', value='data engineering'), Row(text='Principal engineer', value='principal engineer'), Row(text='Senior engineer', value='senior engineer'), Row(text='Business', value='business'), Row(text='Data analyst', value='data analyst'), Row(text='Deputy', value='deputy'), Row(text='Freelancer', value='freelancer'), Row(text='Integration engineer', value='integration engineer'), Row(text='Principal', value='principal'), Row(text='Senior', value='senior')], param='job_family_1', type='Title'), Row(options=[Row(text='All', value=None), Row(text='Bengaluru, Karnataka', value='bU60yXAWrjvgT35R6MPf-A=='), Row(text='Hyderabad, Telangana', value='x9Lr6tqZyzvC-7qSg7eTrg=='), Row(text='Pune, Maharashtra', value='ARFGZy6_wjtD456dv0O

In [47]:
skills_df.printSchema()

root
 |-- chips: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- options: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- param: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- jobs_results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- company_name: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- detected_extensions: struct (nullable = true)
 |    |    |    |-- posted_at: string (nullable = true)
 |    |    |    |-- schedule_type: string (nullable = true)
 |    |    |    |-- work_from_home: boolean (nullable = true)
 |    |    |-- extensions: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- job_highlights: array (nullable = true

In [48]:
from pyspark.sql import functions as F

job_results_df = skills_df.select(F.explode("jobs_results").alias("skills"))

In [49]:
job_results_df.head(1)

[Row(skills=Row(company_name='OptimHire', description='Company Name: Chryselys\n\nPosition: Data Engineer (Experience level 4 to 9 years)\n\nMust-Have Skills: AWS data stack, DevOps tools, Jenkins, CI/CD Gitlab, Kubernetes, ETL tools like AWS Glue, Talend, DBT, Python, SQL, SFDC, RedShift, AWS S3, AppFlow, SFTP, GitHub\n\nLocation: Chennai (For now its remote but once the office starts it would be on-site)\n\nBudget: 18LPA to 25 LPA\n\nInterview: HR Round, 3 Technical Rounds\n\nNote: For this role, we need someone who has experience in pharmaceutical/biotechnology Industry only', detected_extensions=Row(posted_at='23 hours ago', schedule_type='Full-time', work_from_home=True), extensions=['23 hours ago', 'Work from home', 'Full-time'], job_highlights=[Row(items=['Company Name: Chryselys\n\nPosition: Data Engineer (Experience level 4 to 9 years)\n\nMust-Have Skills: AWS data stack, DevOps tools, Jenkins, CI/CD Gitlab, Kubernetes, ETL tools like AWS Glue, Talend, DBT, Python, SQL, SFDC, 

In [50]:
job_results_df.printSchema()

root
 |-- skills: struct (nullable = true)
 |    |-- company_name: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- detected_extensions: struct (nullable = true)
 |    |    |-- posted_at: string (nullable = true)
 |    |    |-- schedule_type: string (nullable = true)
 |    |    |-- work_from_home: boolean (nullable = true)
 |    |-- extensions: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- job_highlights: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- items: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- job_id: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- related_links: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- link: string (nullable = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- thumbnail: string (null

In [51]:
job_results_df = job_results_df.select(
    F.col("skills.description"),
    F.col("skills.job_id"),
    F.col("skills.title"),
    F.col("skills.via")
)

In [52]:
job_results_df.show(4,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
job_results_df.printSchema()

root
 |-- description: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- via: string (nullable = true)



In [54]:
job_results_df.select("title").show(4,truncate=False)

+----------------------------+
|title                       |
+----------------------------+
|Data Engineer               |
|Associate Data Engineer     |
|Engineer II-Data Engineering|
|Data Engineer               |
+----------------------------+
only showing top 4 rows



In [55]:
job_results_df.select("via").show(4,truncate=False)

+-----------------------+
|via                    |
+-----------------------+
|via LinkedIn           |
|via Careers At Boeing  |
|via Verizon            |
|via Mercedes-Benz Group|
+-----------------------+
only showing top 4 rows



In [56]:
job_results_df.select("job_id").show(4,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [57]:
job_results_df = job_results_df.withColumn("id",F.monotonically_increasing_id())

In [58]:
job_results_df = job_results_df.drop("job_id")

In [59]:
job_results_df.show(2,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [60]:
job_results_df = job_results_df.withColumn("role",F.lit("Data Engineer"))

In [61]:
job_results_df.show(2)

+--------------------+--------------------+--------------------+---+-------------+
|         description|               title|                 via| id|         role|
+--------------------+--------------------+--------------------+---+-------------+
|Company Name: Chr...|       Data Engineer|        via LinkedIn|  0|Data Engineer|
|Job Description\n...|Associate Data En...|via Careers At Bo...|  1|Data Engineer|
+--------------------+--------------------+--------------------+---+-------------+
only showing top 2 rows



In [62]:
job_results_df.select(F.regexp_replace("via", "via", "")).show()

+-----------------------------+
|regexp_replace(via, via, , 1)|
+-----------------------------+
|                     LinkedIn|
|            Careers At Boeing|
|                      Verizon|
|          Mercedes-Benz Group|
|                    IKEA Jobs|
|                    AngelList|
|          Caterpillar Careers|
|                     LinkedIn|
|          Fidelity Investm...|
|          Careers.bpglobal...|
+-----------------------------+



In [63]:
job_results_df = job_results_df.withColumn("via",F.regexp_replace("via", "via", ""))

In [64]:
job_results_df.show(5)

+--------------------+--------------------+--------------------+---+-------------+
|         description|               title|                 via| id|         role|
+--------------------+--------------------+--------------------+---+-------------+
|Company Name: Chr...|       Data Engineer|            LinkedIn|  0|Data Engineer|
|Job Description\n...|Associate Data En...|   Careers At Boeing|  1|Data Engineer|
|When you join Ver...|Engineer II-Data ...|             Verizon|  2|Data Engineer|
|TasksResponsibili...|       Data Engineer| Mercedes-Benz Group|  3|Data Engineer|
|Who you are\n\nWe...|       Data Engineer|           IKEA Jobs|  4|Data Engineer|
+--------------------+--------------------+--------------------+---+-------------+
only showing top 5 rows



In [71]:
from datetime import datetime
utc_time = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%s')
utc_time

'2023-04-02-12-30-1680418809'

In [74]:
job_results_df.write.mode("overwrite").parquet(f"./processed/{FILE_NAME}")
